# Marine Highly Migratory Fish

This indicator was used in the Southeast Blueprint 2023.
Code written by Rua Mordecai and Amy Keister

In [51]:
import os
import arcpy

In [52]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [53]:
# define final output name
IndicatorFileName = "MarineHighlyMigratoryFish.tif"

In [54]:
# define raster used for cell size, extent, and snapping
MarineRaster = r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [55]:
# zonation outputs for tuna and blue shark
tuna = r"D:\Blueprint\2023\Indicators\Fish\zonation\tuna\rankmap.tif"
shark = r"D:\Blueprint\2023\Indicators\Fish\zonation\shark\rankmap.tif"
# the data in the species folder used an r script to convert netCDF files to rasters with equal dimensions.
sharkFolder = r"D:\Blueprint\2023\Indicators\Fish\BlueShark"
# file used to match dimensions in tuna zonation mask
exampleFile = r"D:\Blueprint\2023\Indicators\Fish\NOAA_TUNA_OUTPUT\GMIS_BFT25_FH_01_2015.nc.tiff"
# file used to match dimensions in blue shark zonation mask
sharkFile = "cBLUE_SHARK180-330cm_F_2015_01_Earth_Feeding.tif"

### Start analysis

In [56]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

### Prep blue shark data

In [48]:
# Set the workspace to the shark data location
arcpy.env.workspace = sharkFolder

In [49]:
# make a list of indicators 
RList = arcpy.ListRasters() 

In [50]:
# loop through rasters while clipping and reprojecting them
for r in RList:
    outName = "c"+r
    with arcpy.EnvManager(outputCoordinateSystem=MarineRaster, extent=MarineRaster, snapRaster=MarineRaster):
        arcpy.management.Clip(r, "", outName, "Marine_Extent_v1.tif", "1.79e+308", "NONE", "MAINTAIN_EXTENT")

In [37]:
# copy extent and match cell size of input data
with arcpy.EnvManager(outputCoordinateSystem=sharkFile, snapRaster=sharkFile, extent=sharkFile, cellSize=sharkFile, cellAlignment="ALIGN_WITH_PROCESSING_EXTENT"):
    arcpy.management.CopyRaster("Marine_Extent_v1.tif", "sharkMask.tif", '', None, "0", "NONE", "NONE", '', "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [38]:
with arcpy.EnvManager(outputCoordinateSystem=sharkFile, extent=sharkFile):
    arcpy.management.Clip("sharkMask.tif","", "sharkMask2.tif", "", "0", "NONE", "MAINTAIN_EXTENT")

### Prep tuna data

In [ ]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [26]:
# copy extent and match cell size of input data
with arcpy.EnvManager(outputCoordinateSystem=exampleFile, snapRaster=exampleFile, extent=exampleFile, cellSize=exampleFile, cellAlignment="ALIGN_WITH_PROCESSING_EXTENT"):
    arcpy.management.CopyRaster("Marine_Extent_v1.tif", "tunaMask.tif", '', None, "0", "NONE", "NONE", '', "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [10]:
with arcpy.EnvManager(outputCoordinateSystem=exampleFile, extent=exampleFile):
    arcpy.management.Clip("tunaMask.tif","", "tunaMask2.tif", "", "0", "NONE", "MAINTAIN_EXTENT")

### Bring in zonation results

In [7]:
# take tuna zonation results times 100
out_raster = arcpy.sa.Times(tuna, 100); out_raster.save("tuna.tif")

In [8]:
# convert to integer
out_raster = arcpy.sa.Int("tuna.tif"); out_raster.save("tuna1.tif")

In [9]:
# Reclassify into bins and align with Marine extent. 
# Combine everything 0.3 and less b/c tuna didn't have enough info in part of the Atlantic to prioritize that bin
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify('tuna1.tif', "Value", "0 30 1;31 40 2;41 50 3;51 60 4;61 70 5;71 80 6;81 90 7;91 100 8", "DATA"); out_raster.save("tunaBinned.tif")

In [10]:
# take tuna zonation results times 100
out_raster = arcpy.sa.Times(shark, 100); out_raster.save("shark.tif")

In [11]:
# convert to integer
out_raster = arcpy.sa.Int("shark.tif"); out_raster.save("shark1.tif")

In [12]:
# Reclassify into bins and align with Marine extent. 
# Combine everything 0.3 and less b/c tuna didn't have enough info in part of the Atlantic to prioritize that bin
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify('shark1.tif', "Value", "0 30 1;31 40 2;41 50 3;51 60 4;61 70 5;71 80 6;81 90 7;91 100 8", "DATA"); out_raster.save("sharkBinned.tif")

In [20]:
# combine the two zonation outputs by taking the max value of the two
out_raster = arcpy.sa.CellStatistics("tunaBinned.tif;sharkBinned.tif", "MAXIMUM", "NODATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save(IndicatorFileName)

### Finalize indicator



In [57]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 8:
        return '8 = >90th percentile of importance for bluefin tuna and skipjack tuna or blue shark'
    elif v == 7:
        return '7 = >80th-90th percentile of importance'  
    elif v == 6:
        return '6 = >70th-80th percentile of importance'  
    elif v == 5:
        return '5 = >60th-70th percentile of importance'
    elif v == 4:
        return '4 = >50th-60th percentile of importance'
    elif v == 3:
        return '3 = >40th-50th percentile of importance'
    elif v == 2:
        return '2 = >30th-40th percentile of importance'  
    elif v == 1:
        return '1 = ≤30th percentile of importance'
"""

In [58]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\MarineHighlyMigratoryFish.tif'>

In [59]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 8:
		return 88
	if Value == 7:
		return 128
	if Value == 6:
		return 164
	if Value == 5:
		return 192
	if Value == 4:
		return 214
	if Value == 3:
		return 232
	if Value == 2:
		return 245
	if Value == 1:
		return 248
	else:
		return 255
		
def Reclass2(Value):
	if Value == 8:
		return 60
	if Value == 7:
		return 70
	if Value == 6:
		return 86
	if Value == 5:
		return 108
	if Value == 4:
		return 133
	if Value == 3:
		return 160
	if Value == 2:
		return 189
	if Value == 1:
		return 220
	else:
		return 255
		
def Reclass3(Value):
	if Value == 8:
		return 136
	if Value == 7:
		return 154
	if Value == 6:
		return 166
	if Value == 5:
		return 173
	if Value == 4:
		return 178
	if Value == 3:
		return 184
	if Value == 2:
		return 196
	if Value == 1:
		return 217
	else:
		return 255
		
"""

In [60]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\MarineHighlyMigratoryFish.tif'>